In [1]:
import numpy as np

def calculate_route_information(route, distance_matrix, demands, transportation_cost):
    cost = 0.0
    distance = 0
    parcel_value = 0
    vehicle_route = []

    for j in range(len(route) - 1):
        node_i = route[j]
        node_j = route[j + 1]
        distance += distance_matrix[node_i, node_j]
        cost += distance_matrix[node_i, node_j] * transportation_cost

        if node_i != depot:
            parcel_value += demands[node_i]
        vehicle_route.append(node_i)

    vehicle_route.append(route[-1])

    distance += distance_matrix[route[-1], depot]
    cost += distance_matrix[route[-1], depot] * transportation_cost

    return distance, cost, parcel_value, vehicle_route


def tabu_search(distance_matrix, demands, vehicle_capacities, num_vehicles, depot, transportation_cost):
    # Initialize routes and remaining capacity
    routes = []
    remaining_capacity = vehicle_capacities.copy()

    # Initialize remaining_capacity for all vehicles
    for _ in range(num_vehicles - len(vehicle_capacities)):
        remaining_capacity.append(0)

    # Calculate savings
    savings = distance_matrix[:, depot] + distance_matrix[depot] - distance_matrix[:, depot, np.newaxis] - distance_matrix[np.newaxis, depot]

    # Sort savings in descending order
    sorted_indices = np.dstack(np.unravel_index(np.argsort(-savings.ravel()), savings.shape))[0]

    # Iterate over sorted savings
    for indices in sorted_indices:
        i, j = indices

        if i == j or i == depot or j == depot:
            continue

        if len(remaining_capacity) > i and len(remaining_capacity) > depot and remaining_capacity[i] >= demands[i] and remaining_capacity[depot] >= demands[j]:
            found_route = False

            for route in routes:
                if route[-1] == i and route[0] == j:
                    route.insert(-1, j)
                    remaining_capacity[i] -= demands[i]
                    remaining_capacity[depot] -= demands[j]
                    found_route = True
                    break

                if route[-1] == j and route[0] == i:
                    route.insert(-1, i)
                    remaining_capacity[i] -= demands[i]
                    remaining_capacity[depot] -= demands[j]
                    found_route = True
                    break

            if not found_route:
                routes.append([i, j, i])
                remaining_capacity[i] -= demands[i]
                remaining_capacity[depot] -= demands[j]

        if all(capacity == 0 for capacity in remaining_capacity):
            break

    # Tabu Search
    tabu_list = []
    best_distance = sum(distance_matrix[route[j], route[j + 1]] for j in range(len(route) - 1))

    for _ in range(num_iterations):
        candidate_routes = []
        candidate_distances = []

        for i in range(1, len(routes) - 1):
            for j in range(i + 1, len(routes)):
                route_i = routes[i]
                route_j = routes[j]

                for k in range(1, len(route_i) - 1):
                    for l in range(1, len(route_j) - 1):
                        candidate_route_i = route_i[:k] + route_j[l:] + route_j[:l] + route_i[k:]
                        candidate_route_j = route_j[:l] + route_i[k:] + route_i[:k] + route_j[l:]

                        candidate_distance_i = sum(distance_matrix[candidate_route_i[m], candidate_route_i[m + 1]] for m in range(len(candidate_route_i) - 1))
                        candidate_distance_j = sum(distance_matrix[candidate_route_j[m], candidate_route_j[m + 1]] for m in range(len(candidate_route_j) - 1))

                        if candidate_distance_i < best_distance:
                            candidate_routes.append(candidate_route_i)
                            candidate_distances.append(candidate_distance_i)

                        if candidate_distance_j < best_distance:
                            candidate_routes.append(candidate_route_j)
                            candidate_distances.append(candidate_distance_j)

        if len(candidate_routes) == 0:
            break

        best_candidate_index = np.argmin(candidate_distances)
        best_candidate_route = candidate_routes[best_candidate_index]
        best_candidate_distance = candidate_distances[best_candidate_index]

        if best_candidate_route not in tabu_list:
            routes = best_candidate_route
            best_distance = best_candidate_distance
            tabu_list.append(best_candidate_route)
            if len(tabu_list) > tabu_size:
                tabu_list.pop(0)

    # Calculate route information
    total_distance = 0
    total_cost = 0.0
    total_parcel_delivered = sum(demands) - remaining_capacity[depot]

    for i, route in enumerate(routes):
        distance, cost, parcel_value, vehicle_route = calculate_route_information(route, distance_matrix, demands, transportation_cost)
        total_distance += distance
        total_cost += cost

        print(f"Route for driver {i}:")
        print(f" {' -> '.join([str(node) + ' Parcels(' + str(demands[node]) + ')' for node in vehicle_route])}")
        print(f"Distance of the route: {distance} (m)")
        print(f"Parcels Delivered: {parcel_value} (parcels)")
        print(f"Cost of the route: ${cost:.2f}")
        print()

    print(f"Total distance of all routes: {total_distance} (m)")
    print(f"Parcels Delivered: {total_parcel_delivered}/{sum(demands)}")
    print(f"Total cost of all routes: ${total_cost:.2f}")


# Distance matrix
distance_matrix = np.array([[0, 29000, 29000, 46000, 12000, 25000, 29000, 39400, 21000, 28000],
                           [29000, 0, 51000, 18000, 18000, 19000, 23400, 34400, 29000, 25000],
                           [29000, 51000, 0, 68200, 40800, 47000, 52400, 61100, 24700, 51700],
                           [46000, 18000, 68200, 0, 29000, 27800, 15500, 51800, 45900, 29700],
                           [12000, 18000, 40800, 29000, 0, 12300, 20300, 42700, 34800, 15600],
                           [25000, 19000, 47000, 27800, 12300, 0, 12200, 48900, 45900, 1500],
                           [29000, 23400, 52400, 15500, 20300, 12200, 0, 46400, 52600, 5300],
                           [39400, 34400, 61100, 51800, 42700, 48900, 46400, 0, 22300, 39700],
                           [21000, 29000, 24700, 45900, 34800, 45900, 52600, 22300, 0, 33700],
                           [28000, 25000, 51700, 29700, 15600, 1500, 5300, 39700, 33700, 0]])

# Demand and capacity
demands = [0, 9, 11, 8, 12, 10, 7, 13, 11, 9]
vehicle_capacities = [25, 25, 25, 25]
num_vehicles = 4
depot = 0

# Constants
transportation_cost = 0.000497
num_iterations = 100
tabu_size = 5

tabu_search(distance_matrix, demands, vehicle_capacities, num_vehicles, depot, transportation_cost)


Route for driver 0:
 2 Parcels(11) -> 3 Parcels(8) -> 2 Parcels(11)
Distance of the route: 165400 (m)
Parcels Delivered: 19 (parcels)
Cost of the route: $82.20

Route for driver 1:
 1 Parcels(9) -> 3 Parcels(8) -> 1 Parcels(9)
Distance of the route: 65000 (m)
Parcels Delivered: 17 (parcels)
Cost of the route: $32.31

Route for driver 2:
 2 Parcels(11) -> 6 Parcels(7) -> 2 Parcels(11)
Distance of the route: 133800 (m)
Parcels Delivered: 18 (parcels)
Cost of the route: $66.50

Total distance of all routes: 364200 (m)
Parcels Delivered: 88/90
Total cost of all routes: $181.01
